In [137]:
from dataclasses import dataclass
from typing import Annotated, Iterable, TypeAlias
from enum import Enum
from collections import UserList
from itertools import starmap
import numpy as np
from operator import itemgetter

In [138]:
class IntervalBorderTypeEnum(Enum):
    OPEN = "OPEN"
    CLOSED = "CLOSED"


@dataclass
class IntervalBorder:
    type: IntervalBorderTypeEnum
    value: float


@dataclass
class ValueInterval:
    min: IntervalBorder
    max: IntervalBorder


PositiveLessEqualOne: TypeAlias = Annotated[
    float,
    ValueInterval(
        min=IntervalBorder(type=IntervalBorderTypeEnum.OPEN, value=0),
        max=IntervalBorder(type=IntervalBorderTypeEnum.OPEN, value=1),
    ),
]
PositiveProperFractionOrZero: TypeAlias = Annotated[
    float,
    ValueInterval(
        min=IntervalBorder(type=IntervalBorderTypeEnum.OPEN, value=0),
        max=IntervalBorder(type=IntervalBorderTypeEnum.CLOSED, value=1),
    ),
]


@dataclass
class Membership:
    u: PositiveLessEqualOne
    v: PositiveLessEqualOne


@dataclass
class ReferenceParameters:
    a: PositiveLessEqualOne
    b: PositiveLessEqualOne


@dataclass
class DiophantineFuzzyNumber(Membership, ReferenceParameters):
    @property
    def membership_sum(self) -> PositiveLessEqualOne:
        return self.u * self.a + self.v * self.b

    @property
    def dubiety(self) -> PositiveLessEqualOne:
        """aka hesitation = hesitation degree * hesitation degree reference parameter"""
        return 1 - self.membership_sum

    u_index = 0
    v_index = 1
    a_index = 2
    b_index = 3

    def __array__(self):
        return np.array([self.u, self.v, self.a, self.b])

    @staticmethod
    def from_list(arr: list[float]):
        return DiophantineFuzzyNumber(
            u=arr[DiophantineFuzzyNumber.u_index],
            v=arr[DiophantineFuzzyNumber.v_index],
            a=arr[DiophantineFuzzyNumber.a_index],
            b=arr[DiophantineFuzzyNumber.b_index],
        )


# aka t-norm
def conjunction(a: float, b: float) -> float:
    return min(a, b)


# aka t-conorm
def disjunction(a: float, b: float) -> float:
    return max(a, b)


class LinearDiophantineFuzzyRelation:
    data: np.array

    def __init__(
        self,
        Us: Iterable[Iterable[float]] | None = None,
        Vs: Iterable[Iterable[float]] | None = None,
        As: Iterable[Iterable[float]] | None = None,
        Bs: Iterable[Iterable[float]] | None = None,
        array: np.array = None,
    ):
        if Us is not None and Vs is not None and As is not None and Bs is not None:
            self.data = np.fromiter(
                map(
                    lambda line: np.fromiter(
                        map(
                            np.array,
                            starmap(
                                lambda u, v, a, b: DiophantineFuzzyNumber(
                                    u=u, v=v, a=a, b=b
                                ),
                                zip(*line),
                            ),
                        ),
                        dtype=np.dtype((np.float64, 4)),
                    ),
                    zip(Us, Vs, As, Bs),
                ),
                dtype=np.dtype((np.float64, (len(Us[0]), 4))),
            )
        elif array is not None:
            self.data = array
        else:
            self.data = np.array()

    def to_list(self) -> list[list[DiophantineFuzzyNumber]]:
        return list(
            map(
                lambda line: list(map(DiophantineFuzzyNumber.from_list, line)),
                self.data.tolist(),
            ),
        )

    def issubset(self, rel: LinearDiophantineFuzzyRelation) -> bool:
        return (
            self.data[:, :, DiophantineFuzzyNumber.u_index]
            <= rel.data[:, :, DiophantineFuzzyNumber.u_index]
            and self.data[:, :, DiophantineFuzzyNumber.v_index]
            >= rel.data[:, :, DiophantineFuzzyNumber.v_index]
            and self.data[:, :, DiophantineFuzzyNumber.a_index]
            <= rel.data[:, :, DiophantineFuzzyNumber.a_index]
            and self.data[:, :, DiophantineFuzzyNumber.b_index]
            >= rel.data[:, :, DiophantineFuzzyNumber.b_index]
        )

    def union(
        self, rel: LinearDiophantineFuzzyRelation
    ) -> LinearDiophantineFuzzyRelation:
        def unite_diophantine_fuzzy_numbers(ns: np.array) -> np.array:
            print(ns)
            n1 = ns[:4]
            n2 = ns[4:]
            print(n1, n2)
            return np.array(
                [
                    disjunction(
                        *map(itemgetter(DiophantineFuzzyNumber.v_index), (n1, n2))
                    ),
                    conjunction(
                        *map(itemgetter(DiophantineFuzzyNumber.u_index), (n1, n2))
                    ),
                    disjunction(
                        *map(itemgetter(DiophantineFuzzyNumber.a_index), (n1, n2))
                    ),
                    conjunction(
                        *map(itemgetter(DiophantineFuzzyNumber.b_index), (n1, n2))
                    ),
                ]
            )

        return LinearDiophantineFuzzyRelation(
            array=np.apply_along_axis(
                unite_diophantine_fuzzy_numbers, 2, np.dstack((self.data, rel.data))
            )
        )

In [139]:
us1 = [
    [1, 0],
    [0, 1]
    ]
vs1 = [
    [0, 1],
    [1, 0]
    ]
as1 = [
    [1, 0],
    [0, 1]
    ]
bs1 = [
    [0, 1],
    [1, 0]
    ]
us2 = [[0, 1]]
vs2 = [[0, 1]]
as2 = [[0.1, 0.9]]
bs2 = [[0.1, 0.9]]
print(
    LinearDiophantineFuzzyRelation(Us=us1, Vs=vs1, As=as1, Bs=bs1)
    .union(LinearDiophantineFuzzyRelation(Us=us2, Vs=vs2, As=as2, Bs=bs2))
    .to_list()
)

[1.  0.  0.1 0.9 0.  0.  0.1 0.1]
[1.  0.  0.1 0.9] [0.  0.  0.1 0.1]
[0.  1.  0.9 0.1 1.  1.  0.9 0.9]
[0.  1.  0.9 0.1] [1.  1.  0.9 0.9]
[0.  1.  0.9 0.1 1.  1.  0.9 0.9]
[0.  1.  0.9 0.1] [1.  1.  0.9 0.9]
[1.  0.  0.1 0.9 0.  0.  0.  0. ]
[1.  0.  0.1 0.9] [0. 0. 0. 0.]
[[DiophantineFuzzyNumber(a=0.1, b=0.1, u=0.0, v=0.0), DiophantineFuzzyNumber(a=0.9, b=0.1, u=1.0, v=0.0)], [DiophantineFuzzyNumber(a=0.9, b=0.1, u=1.0, v=0.0), DiophantineFuzzyNumber(a=0.1, b=0.0, u=0.0, v=0.0)]]
